In [1]:
from text_mining import load_classes_chebi, delete_files_in_directory, run_text_mining, add_publication, text_prep, CatalysisIE_search
from preprocess_onto import *
from onto_extension_new import preprocess_classes,create_classes_onto
from txt_extract import get_abstract, get_metadata
from CatalysisIE.model import *
from CatalysisIE.utils import *
from pubchempy import get_compounds
import statistics
import json

import pybliometrics
pybliometrics.scopus.init()


def set_config_key(key, value):
    globals()[key] = value
    

C:\Users\smaxbehr\AppData\Local\anaconda3\envs\testenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#ckpt_name = "CatalysisIE/checkpoint/train_demo1_checkpoint-v4.ckpt"
ckpt_name = "CatalysisIE/checkpoint/CV_0.ckpt"
model = BERTSpan.load_from_checkpoint(ckpt_name, model_name=bert_name, train_dataset=[], val_dataset=[], test_dataset=[])

out_dict = {}
label_list = ["Reactant","Product", "Characterization","Reaction","Catalyst","Treatment"]


Global seed set to 12345
Some weights of the model checkpoint at CatalysisIE/pretrained/scibert_domain_adaption were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at CatalysisIE/pretrained/scibert_domain_adaption and ar

In [3]:
get_abstract(None, '10.1007/s10562-023-04565-y', 'springer')

'The mechanism of higher alcohol synthesis (HAS) from syngas on a stepped Rhodium surface was explored using first principles calculations based on density functional theory. Results showed that the activation of CO proceeds most energetically feasible via a sequential hydrogenation towards CH2OH, followed by the C–OH bond cleavage yielding CHx species. Because the initial CO hydrogenation step is highly activated, the cascade of elementary steps toward methane formation is highly favored. The formation of C2 oxygenates toward ethanol production is kinetically favored by CO insertion to CH2, or alternatively, by a lower activation barrier CHO insertion to CH3. On the other hand, the C3 species is formed more preferably by CO rather than CHO insertion to a CH3CH2 fragment, indicating the effect of a more extended carbon structure on the reaction mechanism. The overall reaction mechanism for HAS points to a cycle of CO insertion, hydrogenation, and OH elimination steps. Graphical  (Figur

In [4]:
import csv
with open('./import/DOI-Seed.csv') as csvfile:
    doi_list = list(csv.reader(csvfile))
doi_list

[['10.1007/s10562-023-04565-y'],
 ['10.1016/j.fuel.2023.129772'],
 ['10.1021/ie502425d'],
 ['10.1021/acscatal.3c05351'],
 ['10.1016/j.checat.2024.101010'],
 ['10.1016/j.ijhydene.2024.05.362'],
 ['10.1016/j.ijhydene.2023.12.292'],
 ['10.1021/acscatal.3c05579'],
 ['10.1039/d4ta01179a'],
 ['10.1016/j.cej.2024.148751'],
 ['10.1021/acs.energyfuels.4c00590'],
 ['10.1016/j.fuel.2024.130915'],
 ['10.1016/j.apcata.2024.119636'],
 ['10.1016/j.cej.2024.152502'],
 ['10.1038/s41467-024-50215-1'],
 ['10.1016/j.fuel.2023.130595'],
 ['10.1016/j.apcata.2024.119822'],
 ['10.1002/anie.202401568'],
 ['10.1016/j.mcat.2024.114124'],
 ['10.1016/j.mcat.2023.113270'],
 ['10.1016/j.fuel.2023.127500'],
 ['10.1016/j.apcatb.2023.122840'],
 ['10.31635/ccschem.022.202201930'],
 ['10.1016/j.fuel.2021.122981'],
 ['10.1021/acscatal.3c02534']]

In [11]:
for i in doi_list:
    try:
        abstract = get_abstract(None, i[0], 'NA')
    except:
        print('abstract for doi {} could not be fetched'.format(i))
    
    if abstract != None:
        sents = text_prep(abstract)
        categories, chem_list, reac_dict, sup_cat, abbreviation, raw_entities = CatalysisIE_search(model, sents)

2024-08-29 11:01:35 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-08-29 11:01:36 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | craft   |

2024-08-29 11:01:36 INFO: Using device: cpu
2024-08-29 11:01:36 INFO: Loading: tokenize
2024-08-29 11:01:36 INFO: Done loading processors!
Global seed set to 12345
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.82it/s]


Initialising AllenNLP model ✔   


2024-08-29 11:01:51 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-08-29 11:01:52 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | craft   |

2024-08-29 11:01:52 INFO: Using device: cpu
2024-08-29 11:01:52 INFO: Loading: tokenize
2024-08-29 11:01:52 INFO: Done loading processors!
Global seed set to 12345
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 19.62it/s]
2024-08-29 11:01:53 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-08-29 11:01:54 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | cra

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  6.55it/s]
2024-08-29 11:02:19 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-08-29 11:02:20 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | craft   |

2024-08-29 11:02:20 INFO: Using device: cpu
2024-08-29 11:02:20 INFO: Loading: tokenize
2024-08-29 11:02:20 INFO: Done loading processors!
Global seed set to 12345
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 10.85it/s]
2024-08-29 11:02:22 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-08-29 11:02:23 

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 19.63it/s]
2024-08-29 11:02:46 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-08-29 11:02:47 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | craft   |

2024-08-29 11:02:47 INFO: Using device: cpu
2024-08-29 11:02:47 INFO: Loading: tokenize
2024-08-29 11:02:47 INFO: Done loading processors!
Global seed set to 12345
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 16.35it/s]
2024-08-29 11:02:48 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-08-29 11:02:49 

In [12]:
categories

{'higher alcohol': 'Product',
 'HA': 'Product',
 'CO': 'Reactant',
 'hydrogenation': 'Reaction',
 'modified Fischer-Tropsch synthesis': 'Reaction',
 'CoCu': 'Catalyst',
 'CoFe m-FTS': 'Catalyst',
 'ZrO2': 'Catalyst',
 'syngas': 'Reactant',
 'Cu1Co4 supported on ZrO2-5 catalyst': 'Catalyst',
 'Co1Fe4 supported on ZrO2-10': 'Catalyst'}

In [9]:
print('done')

done
